In [1]:
import numpy as np
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# Import necessary items from Keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, UpSampling2D, ELU
from tensorflow.keras.layers import Conv2DTranspose, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers

In [3]:
def create_model(input_shape, pool_size):
    # Create the actual neural network here
    model = Sequential()
    # Normalizes incoming inputs. First layer needs the input shape to work
    model.add(BatchNormalization(input_shape=input_shape))

    # Below layers were re-named for easier reading of model summary; this not necessary
    # Conv Layer 1
    model.add(Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation=None, name = 'Conv1'))

    # Adding batch normalization before applying non-linear activation function
    model.add(BatchNormalization())

    # Using elu non-linearity since it can produce negative activations and it smoothes slowly
    model.add(ELU(alpha=1.0))

    """
    Note:
    Batch normalization should be done before the activation since the pre-activation outputs are likely to
    adhere to a gaussian distribution and thus benefit from the mean centering and variance scaling to eliminate
    co-variate shift.
    """

    # Conv Layer 2
    model.add(Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = None, name = 'Conv2'))

    # Adding batch normalization before applying non-linear activation function
    model.add(BatchNormalization())

    # Using elu non-linearity since it can produce negative activations and it smoothes slowly
    model.add(ELU(alpha=1.0))
    # However, note that gradient computation will become more expensive, so this may not be a good fit
    # for deployment on resource constrained devices, or for real time use cases that need low latency
    # inference procedures

    # Pooling 1
    model.add(MaxPooling2D(pool_size=pool_size))

    # Conv Layer 3
    model.add(Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = None, name = 'Conv3'))

    # Adding batch normalization before applying non-linear activation function
    model.add(BatchNormalization())

    # Using elu non-linearity since it can produce negative activations and it smoothes slowly
    model.add(ELU(alpha=1.0))

    model.add(Dropout(0.2))
    # Dropout should be applied after batch normalization and non-linear activation

    # Conv Layer 4
    model.add(Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = None, name = 'Conv4'))

    # Adding batch normalization before applying non-linear activation function
    model.add(BatchNormalization())

    # Using elu non-linearity since it can produce negative activations and it smoothes slowly
    model.add(ELU(alpha=1.0))

    model.add(Dropout(0.2))

    # Conv Layer 5
    model.add(Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = None, name = 'Conv5'))

    # Adding batch normalization before applying non-linear activation function
    model.add(BatchNormalization())

    # Using elu non-linearity since it can produce negative activations and it smoothes slowly
    model.add(ELU(alpha=1.0))

    model.add(Dropout(0.2))

    # Pooling 2
    model.add(MaxPooling2D(pool_size=pool_size))

    # Conv Layer 6
    model.add(Conv2D(128, (3, 3), padding='valid', strides=(1,1), activation = None, name = 'Conv6'))

    # Adding batch normalization before applying non-linear activation function
    model.add(BatchNormalization())

    # Using elu non-linearity since it can produce negative activations and it smoothes slowly
    model.add(ELU(alpha=1.0))

    model.add(Dropout(0.2))

    # Conv Layer 7
    model.add(Conv2D(128, (3, 3), padding='valid', strides=(1,1), activation = None, name = 'Conv7'))

    # Adding batch normalization before applying non-linear activation function
    model.add(BatchNormalization())

    # Using elu non-linearity since it can produce negative activations and it smoothes slowly
    model.add(ELU(alpha=1.0))

    model.add(Dropout(0.2))

    # Pooling 3
    model.add(MaxPooling2D(pool_size=pool_size))

    # Upsample 1
    model.add(UpSampling2D(size=pool_size))

    # Deconv 1
    model.add(Conv2DTranspose(64, (3, 3), padding='valid', strides=(1,1), activation = None, name = 'Deconv1'))

    # Adding batch normalization before applying non-linear activation function
    model.add(BatchNormalization())

    # Using elu non-linearity since it can produce negative activations and it smoothes slowly
    model.add(ELU(alpha=1.0))

    model.add(Dropout(0.2))

    # Deconv 2
    model.add(Conv2DTranspose(64, (3, 3), padding='valid', strides=(1,1), activation = None, name = 'Deconv2'))

    # Adding batch normalization before applying non-linear activation function
    model.add(BatchNormalization())

    # Using elu non-linearity since it can produce negative activations and it smoothes slowly
    model.add(ELU(alpha=1.0))

    model.add(Dropout(0.2))

    # Upsample 2
    model.add(UpSampling2D(size=pool_size))

    # Deconv 3
    model.add(Conv2DTranspose(32, (3, 3), padding='valid', strides=(1,1), activation = None, name = 'Deconv3'))

    # Adding batch normalization before applying non-linear activation function
    model.add(BatchNormalization())

    # Using elu non-linearity since it can produce negative activations and it smoothes slowly
    model.add(ELU(alpha=1.0))

    model.add(Dropout(0.2))

    # Deconv 4
    model.add(Conv2DTranspose(32, (3, 3), padding='valid', strides=(1,1), activation = None, name = 'Deconv4'))

    # Adding batch normalization before applying non-linear activation function
    model.add(BatchNormalization())

    # Using elu non-linearity since it can produce negative activations and it smoothes slowly
    model.add(ELU(alpha=1.0))

    model.add(Dropout(0.2))

    # Deconv 5
    model.add(Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = None, name = 'Deconv5'))

    # Adding batch normalization before applying non-linear activation function
    model.add(BatchNormalization())

    # Using elu non-linearity since it can produce negative activations and it smoothes slowly
    model.add(ELU(alpha=1.0))

    model.add(Dropout(0.2))

    # Upsample 3
    model.add(UpSampling2D(size=pool_size))

    # Deconv 6
    model.add(Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = None, name = 'Deconv6'))

    # Adding batch normalization before applying non-linear activation function
    model.add(BatchNormalization())

    # Using elu non-linearity since it can produce negative activations and it smoothes slowly
    model.add(ELU(alpha=1.0))


    # Final layer - only including one channel so 1 filter
    model.add(Conv2DTranspose(1, (3, 3), padding='valid', strides=(1,1), activation = 'sigmoid', name = 'Final'))
    # Using sigmoid for the final layer since the labels are between 0 and 1, which means our output activation
    # should also be in this range

    return model



In [ ]:
def main():
    # Load training images
    train_images = pickle.load(open("./full_CNN_train.p", "rb" ))

    # Load image labels
    labels = pickle.load(open("./full_CNN_labels.p", "rb" ))

    # Make into arrays as the neural network wants these
    train_images = np.array(train_images)
    labels = np.array(labels)

    # Normalize labels - training images get normalized to start in the network
    labels = labels / 255

    # Shuffle images along with their labels, then split into training/validation sets
    train_images, labels = shuffle(train_images, labels)
    # Test size may be 10% or 20%
    X_train, X_val, y_train, y_val = train_test_split(train_images, labels, test_size=0.1)

    # Batch size, epochs and pool size below are all paramaters to fiddle with for optimization
    batch_size = 128
    epochs = 10
    pool_size = (2, 2)
    input_shape = X_train.shape[1:]

    # Create the neural network
    model = create_model(input_shape, pool_size)

    # Using a generator to help the model use less data
    # Channel shifts help with shadows slightly
    datagen = ImageDataGenerator(channel_shift_range=0.2)
    datagen.fit(X_train)

    # Compiling and training the model
    model.compile(optimizer='Adam', loss='mean_squared_error')
    model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=len(X_train)/batch_size,
    epochs=epochs, verbose=1, validation_data=(X_val, y_val))

    # Freeze layers since training is done
    model.trainable = False
    model.compile(optimizer='Adam', loss='mean_squared_error')

    # Save model architecture and weights
    model.save('full_CNN_model.h5')

    # Show summary of model
    model.summary()

if __name__ == '__main__':
    main()

<ipython-input-4-c49fb46255ca>:36: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=len(X_train)/batch_size,


Epoch 1/10
89/89 [==============================] - 1805s 20s/step - loss: 0.0447 - val_loss: 0.3505
Epoch 2/10
89/89 [==============================] - 1827s 20s/step - loss: 0.0125 - val_loss: 0.0170
Epoch 3/10
89/89 [==============================] - 1796s 20s/step - loss: 0.0102 - val_loss: 0.0115
Epoch 4/10
85/89 [===========================>..] - ETA: 1:34 - loss: 0.0092